# Segmenting and Clustering Neighborhoods in Toronto  
### PART - 1 and PART - 2 in brief. Scroll down for PART - 3

In [3]:
#import libraries
import pandas as pd
import numpy as np

#scrape the data from wikipedia and read into a pandas dataframe
wikilink="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
raw_df0=pd.read_html(wikilink, header=0)[0]

# lets check first 5 rows of the dataframe
raw_df0.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# clean up the header
raw_df0.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace= True)

# check only the header
raw_df0.head(0)

,PostalCode,Borough,Neighborhood


Now lets check the number of rows and columns with the 'shape' feature

In [5]:
print('total number of rows = ', raw_df0.shape[0])
print ('number of \'Not assigned\' in Borough = ', raw_df0.Borough.str.count('Not assigned').sum())
print ('number of \'Not assigned\' in Neighborhood = ', raw_df0.Neighborhood.str.count('Not assigned').sum())

total number of rows =  288
number of 'Not assigned' in Borough =  77
number of 'Not assigned' in Neighborhood =  78


Drop 77 rows with 'Not assigned' Borough

In [6]:
#select rows with 'Burough' as 'Not assigned'
indexNames = raw_df0[raw_df0['Borough'] == 'Not assigned' ].index

# now just drop all the selected rows
raw_df1= raw_df0.drop(indexNames)

# basic rule, reset index after rows deleted.
raw_df1.reset_index(drop=True, inplace=True)

#lets check howmany rows are now in the dataframe
raw_df1.shape[0]

211

Perfect!   
Now find the row with Neighborhood as 'Not assigned' and assign the corresponding Borough name to the Neighborhood

In [7]:
raw_df1.loc[raw_df1['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Not assigned


In [8]:
raw_df1.loc[raw_df1['Neighborhood']=='Not assigned','Neighborhood'] = raw_df1[raw_df1['Neighborhood']=='Not assigned']['Borough']
raw_df1.loc[raw_df1['PostalCode']=='M7A']

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Queen's Park


In [9]:
# replace 'Not assigned' with Borough name
raw_df1['Neighborhood'] = raw_df1['Neighborhood'].str.replace('Not assigned', 'Queen\'s Park')

#check the row again
raw_df1.loc[raw_df1['PostalCode']=='M7A']

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Queen's Park


In [10]:
df=(raw_df1.groupby('PostalCode').agg({'Borough':'first','Neighborhood' : ', '.join})
    .reset_index()
   )
df.head(2)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"


In [11]:
#lets check the number of rows in the clean dataframe
print('Cleaned dataframe \'df\' has ',df.shape[0], ' rows')

Cleaned dataframe 'df' has  103  rows


#### --end of part 1--

#### PART - 2 starts here

importing geocoder failed with error.  
as such we will directly work from the csv file with geo coordinates

In [12]:
geolink= 'https://cocl.us/Geospatial_data'
geo_df= pd.read_csv(geolink)
geo_df.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
geo_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
geo_df.head(0)

,PostalCode,Latitude,Longitude


In [14]:
df_coord= df.merge(geo_df, how='inner', left_on=['PostalCode'], right_on=['PostalCode'])
df_coord.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
print('This dataframe now has ', df_coord.shape[0], 'rows and ', df_coord.shape[1], 'columns')

This dataframe now has  103 rows and  5 columns


#### -- end of part 2 --

# PART - 3 Explore and cluster the neighborhoods in Toronto  
Install geopy package (I got error with 'import'. As adviced by coders in Stackoverflow, 'install' works though page turn messy)

In [16]:
!conda install -c conda-forge geopy --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Anaconda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2018.03.07 |                0         155 KB
    certifi-2018.11.29         |           py37_0         146 KB
    openssl-1.1.1a             |       he774522_0         5.7 MB
    ------------------------------------------------------------
                                           Total:         6.0 MB

The following packages will be UPDATED:

  anaconda                                    custom-py37_1 --> 2018.12-py37_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2019.6.1~ --> pkgs/main::ca-certificates-2018.03.07-0
  certifi             conda-forge::certifi-2019.6.16-py37_1 --> pkgs/main::certifi-2018.11.29-p

In [17]:
#first import necessary librarires
import folium
import requests
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim #converts an address to latitude longitude
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### We will select only boroughs that contain the word Toronto.

In [18]:
# lets make a dataframe for Toronto boroughs
df_Tor=df_coord[df_coord['Borough'].str.contains('Toronto', case=False)].reset_index(drop=True)
df_Tor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


##### We need to define a user_agent, inorder to define a geopy instance.  
Assume 'Tor_explore' as our user_agent, lets define

In [19]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent= 'Tor_explore')
location= geolocator.geocode(address)
latitude= location.latitude
longitude= location.longitude
print('Geo coordinates of Torento are {}, {}.'.format(latitude, longitude))

Geo coordinates of Torento are 43.653963, -79.387207.


##### Create a map of Toronto city with neighborhoods superimposed on top

In [20]:
#create map with latitude and longitude
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

#add markers
for lat, lng, borough, neighborhood in zip(df_Tor['Latitude'], df_Tor['Longitude'], df_Tor['Borough'], df_Tor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius= 5,
        popup=label,
        color= 'green',
        fill= True,
        fill_color= '3186cc',
        fill_opacity= 0.7,
        parse_html= False).add_to(map_Toronto)

map_Toronto

#### OK, time for Foursquare API to explore the neighborhood and do the segmenting.

In [21]:
CLIENT_ID= 'WEFVVVVMJITMPKN0JGOK0ZSUIXV4WXLFSBWTMY12OFMB4ON4'
CLIENT_SECRET= 'LTV2Z3HQ3QKWSN40J31NDBK51A05RYINW3XP2BNLXL3RTVQW'
VERSION= '20180605'
LIMIT=200
print('Forsquare Credentilas:\n', 'CLIENT ID: ' +CLIENT_ID, '\nCLIENT SECRET: ' +  CLIENT_SECRET)

Forsquare Credentilas:
 CLIENT ID: WEFVVVVMJITMPKN0JGOK0ZSUIXV4WXLFSBWTMY12OFMB4ON4 
CLIENT SECRET: LTV2Z3HQ3QKWSN40J31NDBK51A05RYINW3XP2BNLXL3RTVQW


### Lets begin exploration
##### We will create a function to explore all the neighborhoods in Toronto

In [22]:
def GetNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        #creating API request url
        url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        #making first request
        results=requests.get(url).json()["response"]['groups'][0]['items']
        
        #return only relevant informations
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name'])for v in results])
        nearby_venues=pd.DataFrame([item for venues_list in venues_list for item in venues_list])
        nearby_venues.columns = ['Neighborhood',
                                'Neighborhood Latitude',
                                'Neighborhood Longitude',
                                'Venue',
                                'Venue Latitude',
                                'Venue Longitude',
                                'Venue Category']
    return(nearby_venues)
                                       

In [23]:
df_Tor.shape

(38, 5)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called Toronto_venues

In [24]:
Toronto_venues= GetNearbyVenues(names=df_Tor['Neighborhood'],
                               latitudes=df_Tor['Latitude'],
                               longitudes=df_Tor['Longitude']
                               )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

#### Let see the size of the DataFrame

In [25]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1706, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


#### Lets check how many venues were returned for each neighborhood

In [26]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,86,86,86,86,86,86
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


#### Now lets find out how many unique categories can be extracted from all the returned venues

In [27]:
print('There are {} unique categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 236 unique categories.


### Analyze each neighborhood

In [28]:
# onehot encoding 
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to the dataframe
Toronto_onehot['Neighborhood']= Toronto_venues['Neighborhood']

#move Neighborhood column as the first column
shift_column= [Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[:-1])
Toronto_onehot= Toronto_onehot[shift_column]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Lets check the new dataframe size

In [29]:
Toronto_onehot.shape

(1706, 236)

#### Lets group the records by Neighborhood by the mean of the frequency of the occurrence of each category

In [30]:
Toronto_grouped= Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.000000,0.030000,0.0000,0.000000,0.000000,0.0000,0.000000,0.020000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.020000,0.000000,0.000000,0.00,0.030000,0.010000,0.00,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.080000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.0100

#### Check the new size

In [31]:
Toronto_grouped.shape

(38, 236)

#### Lets print each neighborhood with top five most common venues

In [32]:
num_top_venues=5
for hood in Toronto_grouped['Neighborhood']:
    print("----""+hood+""----")
    temp= Toronto_grouped[Toronto_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----+hood+----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2   Steakhouse  0.04
3          Bar  0.04
4   Restaurant  0.03


----+hood+----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2          Café  0.04
3   Cheese Shop  0.04
4      Beer Bar  0.04


----+hood+----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3             Gym  0.05
4             Bar  0.05


----+hood+----
                venue  freq
0  Light Rail Station  0.12
1          Comic Shop  0.06
2              Garden  0.06
3       Auto Workshop  0.06
4      Farmers Market  0.06


----+hood+----
             venue  freq
0   Airport Lounge  0.13
1  Airport Service  0.13
2      Coffee Shop  0.07
3            Plane  0.07
4              Bar  0.07


----+hood+----
         venue  freq
0  Coffee Shop  0.07
1          Pub  0.05
2         Café  0.05
3       Bakery  0.05
4   Restaurant  0.05


----+hood+----
                       venue  freq
0 

#### Lets put them into a dataframe
Lets write a function to sort the venues in descending order

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories= row.iloc[1:]
    row_categories_sorted= row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [34]:
num_top_venues=10
indicators = ['st', 'nd', 'rd']

#create columns based on number of top 10 venues
columns= ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Cosmetics Shop,Asian Restaurant,Burger Joint,Hotel,Thai Restaurant,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Cheese Shop,Farmers Market,Steakhouse,Beer Bar,Bakery,Belgian Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym,Intersection,Convenience Store,Pet Store,Gym / Fitness Center,Nightclub,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Farmers Market,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Harbor / Marina,Airport Terminal,Sculpture Garden,Coffee Shop,Bar,Boat or Ferry,Plane,Airport Gate


#### Clustering Neighborhhods
Let use k-means and contain the neighborhoods in 5 cluster

In [35]:
kclusters = 6

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [36]:
# adding clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_Tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Pub,Spa,Bookstore,Brewery,Bubble Tea Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Pizza Place,Pet Store,Board Shop,Brewery,Burger Joint,Sandwich Place,Burrito Place,Pub,Park,Sushi Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,5,Bus Line,Park,Lawyer,Swim School,Wings Joint,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


### Now lets create the map with markers

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining and labeling the Clusters
#### we will explore each cluster and find out the prominent venues of each category

In [38]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Trail,Pub,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
9,Central Toronto,0,Pub,Coffee Shop,Pizza Place,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Restaurant,Liquor Store,Fried Chicken Joint


##  Cluster - 1: Shopping Area
#### Cluster 1 most common venues are Health food store and pub. the area has mix of categories - mostl likely a shopping place in the city
Lets check Cluster 2 next

In [39]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Pub,Spa,Bookstore,Brewery,Bubble Tea Shop
2,East Toronto,1,Pizza Place,Pet Store,Board Shop,Brewery,Burger Joint,Sandwich Place,Burrito Place,Pub,Park,Sushi Restaurant
3,East Toronto,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place,Cheese Shop
5,Central Toronto,1,Clothing Store,Breakfast Spot,Dog Run,Food & Drink Shop,Hotel,Park,Sandwich Place,Grocery Store,Gym,Electronics Store
6,Central Toronto,1,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Mexican Restaurant,Diner,Miscellaneous Shop,Dessert Shop,Park,Chinese Restaurant
7,Central Toronto,1,Pizza Place,Dessert Shop,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Café,Gym,Deli / Bodega,Greek Restaurant
11,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Pizza Place,Pub,Pharmacy,Bakery,Park,Italian Restaurant,Pet Store
12,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym
13,Downtown Toronto,1,Coffee Shop,Café,Bakery,Park,Breakfast Spot,Mexican Restaurant,Theater,Gym / Fitness Center,Pub,Performing Arts Venue
14,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Tea Room,Diner,Sporting Goods Shop,Bookstore,Fast Food Restaurant


## Cluster - 2: Coffee Shops and Restaurants
#### Most prominet in the area is coffeeshops
Next Cluster 3

In [84]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Pub,Spa,Bookstore,Brewery,Bubble Tea Shop
2,East Toronto,2,Pizza Place,Pet Store,Board Shop,Brewery,Burger Joint,Sandwich Place,Burrito Place,Pub,Park,Sushi Restaurant
3,East Toronto,2,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place,Cheese Shop
5,Central Toronto,2,Clothing Store,Breakfast Spot,Dog Run,Food & Drink Shop,Hotel,Park,Sandwich Place,Grocery Store,Gym,Electronics Store
6,Central Toronto,2,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Mexican Restaurant,Diner,Miscellaneous Shop,Dessert Shop,Park,Chinese Restaurant
7,Central Toronto,2,Pizza Place,Dessert Shop,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Café,Gym,Deli / Bodega,Greek Restaurant
9,Central Toronto,2,Pub,Coffee Shop,Pizza Place,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Restaurant,Liquor Store,Fried Chicken Joint
11,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Pizza Place,Pub,Pharmacy,Bakery,Park,Italian Restaurant,Pet Store
12,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym
13,Downtown Toronto,2,Coffee Shop,Café,Bakery,Park,Breakfast Spot,Mexican Restaurant,Theater,Gym / Fitness Center,Pub,Performing Arts Venue


## Cluster - 3: Coffee shops and restaurants
#### Coffee shops are most common venue here as well. an increased number of foreign cuisine from clsuter 2 here
Next Cluster 4

In [40]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Garden,Wings Joint,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


## Cluster - 4: Garden Place
#### This could be the city central. most common avenue is the Garden
Next Cluster 5

In [41]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


## Cluster - 5: Park
#### Cluster 4 and 5 are of the same charecteristic
Next Cluster 6

In [43]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,5,Bus Line,Park,Lawyer,Swim School,Wings Joint,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


## Cluster - 6: Bus Line area
#### probably the bus hub in the city

### -- end of analysis--